In [1]:
!pip install transformers gradio
!pip install opencc-python-reimplemented     #  簡體字 <==> 繁體字

這個時候不應有 >。


In [2]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM   # 分詞器   模型載入
import torch  #  pytorch 協助 GPU 算利運算 及 深度學習處理
import opencc  # 導入 opencc 庫，用於進行簡體中文到繁體中文的轉換
# 載入模型和分詞器
model_name = "Qwen/Qwen2-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 檢查是否有可用的 GPU，如果有則將模型載入到 GPU 上，否則使用 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
# 建立一個 OpenCC 轉換器，用於將簡體中文轉換為繁體中文
converter = opencc.OpenCC('s2t')  # 's2t' 表示從簡體轉為繁體
def answer_question(question):
    # 將輸入的問題進行編碼，並生成 attention_mask
    inputs = tokenizer(question, return_tensors="pt", padding=True)
    # 將編碼後的輸入向量傳到到與模型相同的 device 上（GPU 或 CPU）
    inputs = {key: value.to(device) for key, value in inputs.items()}
    # 設定 pad_token_id 為 eos_token_id，以避免在處理填充時出現混淆
    pad_token_id = tokenizer.eos_token_id
    # 使用 LLM 模型生成答案，這裡設置 max_new_tokens 來限制生成的最大長度
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=256,
        pad_token_id=pad_token_id
    )
    # 解碼生成的答案，將模型輸出的 token 轉換為人類可讀的內容
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # 將簡體中文答案轉換為繁體中文
    answer_traditional = converter.convert(answer)
    return answer_traditional

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
import gradio as gr
import requests
def call_llm_api(question):
    # 設定外部 API 的 URL
    api_url = "http://127.0.0.1:5000/qa"  # 此處替換為實際的 Flask API 地址
    # 向外部 API 發送請求並獲取回傳答案
    response = requests.post(api_url, json={"text": question})
    if response.status_code == 200:
        # 解析並回傳答案
        answer = response.json().get("answer", "無法獲取答案")
        return answer
    else:
        return "發生錯誤: 無法從 API 獲取答案。"
# 使用 Gradio 建立介面
iface = gr.Interface(
    fn=call_llm_api,  # 綁定處理函數
    inputs="text",  # 設置輸入類型為文字
    outputs="text",  # 設置輸出類型為文字
    title="繁體中文 LLM QA 問答系統",  # 設置介面標題
    description=f"輸入您的問題，獲取 {model_name} 模型的回答。",  # 設置介面描述
)
# 啟動 Gradio 介面並提供外部連結
iface.launch(share=True)

from flask import Flask, request, jsonify
# 建立一個 Flask 應用
app = Flask(__name__)
# 定義一個路由 '/qa'，使用 POST 方法來處理問答請求
@app.route('/qa', methods=['POST'])
def qa():
    # 檢查請求中是否包含 'text' 字段，如果沒有則回傳錯誤訊息
    if 'text' not in request.json:
        return jsonify({"error": "未提供文本"}), 400  # 400 表示客戶端錯誤
    # 從請求中提取問題文字
    question = request.json['text']
    # 呼叫 answer_question 函數來生成答案
    answer = answer_question(question)
    # 將答案以 JSON 格式回傳
    return jsonify({"answer": answer})
# 啟動 Flask 服務，監聽所有網絡接口，並設定端口為 5000
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://2c2cd4a08d3e44b0c7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.88.35:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Sep/2024 14:02:03] "POST /qa HTTP/1.1" 200 -
127.0.0.1 - - [08/Sep/2024 14:02:32] "POST /qa HTTP/1.1" 200 -
